<a href="https://colab.research.google.com/github/bibowskii/LLM-T5-small-summarizer/blob/main/t5_trainer_round_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

preprocessing the text
cleaning
unused and unnecessary




In [ ]:
from google.colab import drive
import time
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext




tokenizing using a pretrained model and loading our pretrained model


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-small')
modelT5 = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/nlp_project/model')

def tokenize(text):
    cleaned_text = text

    tokenized_text = tokenizer.encode(cleaned_text, return_tensors="pt")
    return tokenized_text

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


loading the dataset and splitting

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset


trainingSet = load_dataset("woshityj/xsum_dataset", split="train" )


#checking the data
print(trainingSet[0])

README.md:   0%|          | 0.00/565 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/5.62M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/5.49M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3750 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3750 [00:00<?, ? examples/s]

{'document': 'The bridge has been closed since midnight with initial guidance saying it would be shut for at least 24 hours.The minister has now said it would need at least three weeks to repair the bridge and safety was "paramount".He said a full travel plan was being prepared, including extra trains and buses and possibly a ferry.Early on Friday morning, 11-mile tailbacks were reported approaching the Kincardine Bridge, the alternative route across the Forth from Fife to Edinburgh.Heavy congestion was reported on the same routes on Friday evening.At a media conference, engineers said a 20mm-wide crack in a truss under the southbound carriageway close to the bridge\'s north tower could not have been predicted and happened quickly.They said continuing to allow traffic to use the bridge would "increase the risk of causing extensive secondary damage to the structure".Follow our Scotland Live page for regular updatesWhy is the Forth Road Bridge closed?The troubles of the Forth Road Bridge

function to tokenize the dataset using the preloaded tokenizer

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelT5.to(device)
print(device)

cuda


In [ ]:
def tokenize_dataset(dataset):
    inputs = tokenizer(
        dataset['document'],
        return_tensors="pt",
        padding="max_length",  # Pad to max_length (only specified once)
        truncation=True,       # Truncate to max_length
        max_length=1000        # Specify max_length for the article
    )
    targets = tokenizer(
        dataset['summary'],
        return_tensors="pt",
        padding="max_length",  # Pad to max_length (only specified once)
        truncation=True,       # Truncate to max_length
        max_length=200         # Specify max_length for the highlights
    )

    inputs['labels'] = targets['input_ids']
    inputs['decoder_attention_mask'] = targets['attention_mask']
    return inputs


applying tokenization and mapping the input to the output


In [ ]:
tokenized_trainingSet = trainingSet.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

formatting the dataset

In [ ]:
tokenized_trainingSet.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])

verifying the datasets

In [ ]:
example = tokenized_trainingSet[0]
print(example.keys())
print(tokenizer.decode(example['input_ids']))
print(tokenizer.decode(example['labels']))

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])
The bridge has been closed since midnight with initial guidance saying it would be shut for at least 24 hours.The minister has now said it would need at least three weeks to repair the bridge and safety was "paramount".He said a full travel plan was being prepared, including extra trains and buses and possibly a ferry.Early on Friday morning, 11-mile tailbacks were reported approaching the Kincardine Bridge, the alternative route across the Forth from Fife to Edinburgh.Heavy congestion was reported on the same routes on Friday evening.At a media conference, engineers said a 20mm-wide crack in a truss under the southbound carriageway close to the bridge's north tower could not have been predicted and happened quickly.They said continuing to allow traffic to use the bridge would "increase the risk of causing extensive secondary damage to the structure".Follow our Scotland Live page for regular updatesWhy is th

data loader to test in batches

In [ ]:
from torch.utils.data import DataLoader
trainingLoader = DataLoader(tokenized_trainingSet, batch_size=4, shuffle= False, num_workers=2)


trainer


In [ ]:
from transformers import AdamW
from torch.nn import CrossEntropyLoss

def train_model(model, trainLoader, optimizer, device):
  model.train()
  total_loss = 0
  for batch in trainLoader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    optimizer.zero_grad()
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    total_loss += loss.item()
    loss.backward()
    optimizer.step()

  return total_loss / len(trainLoader)

epochs

In [ ]:
checkpoint_path = "/content/drive/MyDrive/LLMmodels/round2/epoch={epoch}"
num_epochs = 3
optimizer = AdamW(modelT5.parameters(), lr=5e-5)
for epoch in range(num_epochs):
  start_time = time.time()
  print(f"Epoch {epoch+1}/{num_epochs}")
  train_loss = train_model(modelT5, trainingLoader, optimizer, device)
  print(f"Train Loss: {train_loss}")
  modelT5.save_pretrained(checkpoint_path.format(epoch=epoch+4))
  end_time = time.time()
  excution_time = (end_time - start_time)/60
  print('time taken for this epoch:  {:.2f} minutes' .format(excution_time))

Epoch 1/3


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Train Loss: 0.41398891493082046
time taken for this epoch:  48.38 minutes
Epoch 2/3
Train Loss: 0.39881168570518494
time taken for this epoch:  48.45 minutes
Epoch 3/3
Train Loss: 0.38823690070907274
time taken for this epoch:  48.44 minutes
